In [1]:
import open3d as o3d
import cl
import utils as ut
import numpy as np
from skimage.morphology import binary_dilation
import proc3d
import json
from PIL import Image
from utils import *
import glob
import os
%matplotlib widget 
#%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [6]:
class space_carving():
    def __init__(self, dataset_path):
        self.images = self.load_images(os.path.join(dataset_path, 'imgs'))
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_images(self,path):                                                                                                                                         
        imgs = []
        img_files = glob.glob(os.path.join(path, '*.png')) #get all .png files from folder path
        assert len(img_files) != 0,"Image list is empty."
        for i in sorted(img_files):                                                                                                                                     
            img = Image.open(i)                                                                                                                                      
            imgs.append(img.copy())                                                                                                                                     
            img.close()                                                                                                                                                                                                                                                   
        return imgs

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1
        
        print(nx,ny,nz)

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        self.space_carve(self.images[idx], self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [7]:
class space_carving_2():
    def __init__(self, dataset_path):
        self.img_files = sorted (glob.glob(os.path.join(dataset_path, 'imgs', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        #self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.bbox = json.load(open(os.path.join(dataset_path, '/home/pico/uni/romi/scanner_cube/bbox_min_max.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_image(self,idx):                                                                                                                                         
        img = Image.open(self.img_files[idx])                                                                                                                                      
        cp = img.copy()                                                                                                                                 
        img.close()                                                                                                                                                                                                                                                   
        return cp

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_image(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, im, rt):
        mask = get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [96]:
#data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/000'
#spc = space_carving(data_path)

#for i in range(180):
#    spc.carve(i)

# for creating a responsive plot
%matplotlib widget
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
  
x,y,z = np.where(spc.sc.values()[:64,:64,:128]==1)
  
# creating figure
fig = plt.figure()
ax = Axes3D(fig)

#ax.set_xlim3d(0, 70)
#ax.set_ylim3d(0, 70)
#ax.set_zlim3d(0, 152)
  
# creating the plot
plot_geeks = ax.scatter(x, y, z, color='green',s=1)

x,y,z = np.where(spc.sc.values()==0)
#plot_geeks = ax.scatter(x, y, z, color='red',s=1)
  
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
ax.grid()
  
# displaying the plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
np.histogram(spc.sc.values(), bins=3)

(array([2251293,    9798,   17997]),
 array([-1.        , -0.33333333,  0.33333333,  1.        ]))

In [94]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/006'#'/home/pico/uni/romi/rl_sony/scanner_cube/arabidopsis_im_bigger/001'
spc = space_carving_2(data_path)

INFO - 2021-09-20 23:38:01,409 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


In [12]:
spc.carve(160)
print(np.histogram(spc.sc.values(), bins=3)[0])

[471206  81731 129239]


In [101]:
print(np.histogram(spc.sc.values(), bins=3)[0])

[      0 2279088       0]


In [89]:
ant = 0
for i in range(0,180,40):
    spc.carve(i)
    h = np.histogram(spc.sc.values(), bins=3)
    print(i,h[0],h[0][0]-ant)
    ant=h[0][0]

0 [449666 177999  54511] 449666
40 [523290 147790  11096] 73624
80 [619673  56545   5958] 96383
120 [663191  14859   4126] 43518
160 [666321  12993   2862] 3130


In [95]:
pos = np.random.randint(180, size=10)

for i in pos:
    spc.carve(i)
    print(i,np.histogram(spc.sc.values(), bins=3)[0])


76 [361287 275399  45490]
140 [612176  34789  35211]
39 [633134  22584  26458]
29 [640268  19192  22716]
96 [656108  16650   9418]
172 [664441  12736   4999]
2 [667348  12501   2327]
23 [667539  12427   2210]
100 [667793  12229   2154]
89 [667906  12194   2076]


In [13]:
spc.sc.values().shape

(66, 68, 152)

In [41]:
data_path = '/home/pico/uni/romi/scanner_cube/arabidopsis_im_bigger/'
gdeltas = []
for plantn in range(180):
    spc = space_carving_2(os.path.join(data_path,str(plantn).zfill(3)))
    for i in range(10):
        deltas = []
        pos = np.random.randint(180, size=10)
        n_ant = 0
        for p in pos:
            spc.carve(p)
            espaces = np.histogram(spc.sc.values(), bins=3)[0][0]
            delta = espaces-n_ant
            deltas.append(delta)
            n_ant = espaces
        gdeltas.append(deltas)

INFO - 2021-09-20 20:50:32,082 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)
INFO - 2021-09-20 20:50:41,206 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)
INFO - 2021-09-20 20:50:50,239 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


[Open3D WARNING] Read PLY failed: unable to open file: /home/pico/dataset_arabidopsis3d/arabidopsis_2_gt.ply


INFO - 2021-09-20 20:50:59,211 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)
INFO - 2021-09-20 20:51:08,261 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


[Open3D WARNING] Read PLY failed: unable to open file: /home/pico/dataset_arabidopsis3d/arabidopsis_4_gt.ply


INFO - 2021-09-20 20:51:17,354 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


...

[Open3D WARNING] Read PLY failed: unable to open file: /home/pico/dataset_arabidopsis3d/arabidopsis_177_gt.ply


INFO - 2021-09-20 21:19:32,986 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


[Open3D WARNING] Read PLY failed: unable to open file: /home/pico/dataset_arabidopsis3d/arabidopsis_178_gt.ply


INFO - 2021-09-20 21:19:42,974 - __init__ - build program: kernel 'carve' was part of a lengthy source build resulting from a binary cache miss (0.22 s)


[Open3D WARNING] Read PLY failed: unable to open file: /home/pico/dataset_arabidopsis3d/arabidopsis_179_gt.ply


In [70]:
np.histogram(spc.sc.values(), bins=3)[0][0]

668700

In [42]:
dt = np.array(gdeltas)

In [43]:
dt.shape

(1800, 10)

In [ ]:
d

In [44]:
np.mean(dt,axis=0)

array([6.43290162e+05, 1.21927844e+04, 6.14337056e+03, 3.09219778e+03,
       1.49961389e+03, 8.63421111e+02, 4.80713333e+02, 2.96538889e+02,
       2.59505000e+02, 1.81798889e+02])

In [45]:
np.min(dt,axis=0)

array([273639,      0,      0,      0,      0,      0,      0,      0,
            0,      0])

In [46]:
np.max(dt,axis=0)

array([670506, 285669, 257374, 262049, 126021,  92612,  80575,  34797,
        69688,  66288])

In [47]:
np.std(dt,axis=0)

array([80197.22686439, 42857.78320588, 26500.97885655, 15687.89702874,
        8040.58085674,  5282.99535201,  3449.51932385,  1714.96226251,
        2139.27237344,  1776.36642666])

In [62]:
def calculate_position(init_state,steps):
        n_positions = 180
        n_pos = init_state + steps
        if n_pos>(n_positions-1):
            n_pos -= n_positions
        elif n_pos<0:
            n_pos += n_positions
        return n_pos

In [101]:
rel_pos = 0
p_bias = 179
abs_pos = calculate_position(rel_pos,p_bias)
print(abs_pos)

179
